In [1]:
deployment = "gpt4"

# 文本向量化（embedding)

In [11]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(deployment="embedding",chunk_size=1)
sentence1 = "我是一名软件工程师"
sentence2 = "小张从事法律工作"
sentence3 = "我是一名程序员"

embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [4]:
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [12]:
import numpy as np
print(np.dot(embedding1,embedding2))
print(np.dot(embedding2,embedding3))
print(np.dot(embedding1,embedding3))

0.7987662070931474
0.7992461569419606
0.9592105615669977


# 处理大文本

## 切割

In [2]:
def split_file_into_chunks(file_path, chunk_size):
    chunks = []
    with open(file_path, 'r') as file:
        while True:
            chunk = file.read(chunk_size)
            if not chunk:
                break
            chunks.append(chunk)
    return chunks

In [19]:
split_file_into_chunks("spotmax_intro.txt",100)

['SpotMax\n打破成本约束，释放云端创造力\n\n构建大规模云端系统的挑战\n云计算平台“按需获取”的特性使得用户可以随时获取所需计算资源，实现计算资源与业务规模的动态匹配。云计算时代计算资源已不再是构建',
 '大规模系统的瓶颈，多数情况下成本成为了企业在构建大规模应用时的最大挑战。\n\n什么是 SpotMax?\nSpotMax是一个利用弹性计算资源进行云端系统成本优化的体系（包括相关产品及解决方案）。Spot',
 'Max旨在帮助用户使用云平台提供的弹性资源（如：AWS Spot Instance）在保证系统可用性，甚至获得更高可用性的前提下，实现成本的最大化节省。SpotMax来自于Mobvista降低成本的成',
 '功实践。\nSpotMax由以下三个部分组成：\nMaxArch：构建高可用性，高弹性（Resilient）架构，可以允许系统在不影响业务的前提下更充分的使用具有成本优势的高弹性计算资源（Spot Ins',
 'tance）。\nMaxChaos：利用混沌工程可以有效的评估系统的容错能力，其评估结果将成为MaxGroup自动规划合理弹性集群的依据。实现MaxArch的持续改进及评估。\nMaxGroup：根据服务',
 '工作模式和运行特征，优化主机群集。智能弹性集群管理可以有效减少弹性计算资源波动（如：竞价实例的中断）带来的集群服务能力变化。并在面向成本优化构建的混合机型集群最大发挥集群不同机型的服务能力\n以上各部分',
 '将以咨询服务和软件产品的形式提供。\nMaxGroup 核心功能\nMaxGroup可以根据服务的工作模式和运行时特征，对主机集群的组成提出优化建议（如预留实例、按需实例与Spot型实例的比率，Spot实',
 '例池等），并估算出此优化节省的成本。\n用户确认优化集群方案后，自动实现新集群的构建或原有集群的构成的直接优化(安全替换现有集群的运行实例)。\n通过对集群中Spot instance中断的预先补偿, 降',
 '低中断带来的服务器容量变化带来的服务能力影响，实现服务及数据的提前迁移。\nMaxGroup将根据现货价格和终止率的变化持续优化集群结构。 \n支持智能的动态负载均衡，由于集群为多机型混合，不同机型的处理',
 '能力有差异，智能的动态负载均衡将实现按处理能力的负载分配，实现集群资源

## 向量化 

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings

def embed(chunks): 
    embedding = OpenAIEmbeddings(deployment="embedding",chunk_size=1)
    return [embedding.embed_query(chunk) for chunk in chunks]
        

In [4]:
chunks = split_file_into_chunks("spotmax_intro.txt",100)
embeddeds = embed(chunks)

## 向量检索

In [5]:
def find_k_largest_indices(input_list, k):
    sorted_indices = sorted(range(len(input_list)), key=lambda i: input_list[i], reverse=True)
    return sorted_indices[:k]

def search(chunks,embeddeds,top_k, txt):
    embedding = OpenAIEmbeddings(deployment="embedding",chunk_size=1)
    embedded_txt = embedding.embed_query(txt)
    distances = [np.dot(embedded_txt,embedded) for embedded in embeddeds]
    ret_idx = find_k_largest_indices(distances, top_k)
    return [chunks[i] for i in ret_idx]

In [29]:
search(chunks,embeddeds,2, "提高系统可用性")

['工作模式和运行特征，优化主机群集。智能弹性集群管理可以有效减少弹性计算资源波动（如：竞价实例的中断）带来的集群服务能力变化。并在面向成本优化构建的混合机型集群最大发挥集群不同机型的服务能力\n以上各部分',
 '功实践。\nSpotMax由以下三个部分组成：\nMaxArch：构建高可用性，高弹性（Resilient）架构，可以允许系统在不影响业务的前提下更充分的使用具有成本优势的高弹性计算资源（Spot Ins']

In [30]:
search(chunks,embeddeds,2, "省钱")

['SpotMax\n打破成本约束，释放云端创造力\n\n构建大规模云端系统的挑战\n云计算平台“按需获取”的特性使得用户可以随时获取所需计算资源，实现计算资源与业务规模的动态匹配。云计算时代计算资源已不再是构建',
 '大规模系统的瓶颈，多数情况下成本成为了企业在构建大规模应用时的最大挑战。\n\n什么是 SpotMax?\nSpotMax是一个利用弹性计算资源进行云端系统成本优化的体系（包括相关产品及解决方案）。Spot']

In [31]:
search(chunks,embeddeds,2, "什么是MaxGroup")

['能力有差异，智能的动态负载均衡将实现按处理能力的负载分配，实现集群资源利用的最大化。\nMaxGroup可以根据工作负载变化模式（缓慢变化或急剧变化）和业务需求为系统自动伸缩选择合适的实例类型。 \nMa',
 '低中断带来的服务器容量变化带来的服务能力影响，实现服务及数据的提前迁移。\nMaxGroup将根据现货价格和终止率的变化持续优化集群结构。 \n支持智能的动态负载均衡，由于集群为多机型混合，不同机型的处理']

In [8]:
import openai
import numpy as np
def anwser_question_with_doc(question, chunks, embeddeds):
    relevent_chunks = search(chunks,embeddeds,2, question)
    prompt = """
    仅通过总结以下的文字片段回答用户问题, 注意保持回答的语言通顺（字数在30字以内）
    --- 
    """
    for rchunks in relevent_chunks:
        prompt = prompt + "\n'" + rchunks + "'"
    
    response = openai.ChatCompletion.create(
    engine=deployment, # engine = "deployment_name".
    messages=[
            {"role": "system", "content": prompt},   
            {"role": "user", "content": question}
        ],
        temperature = 0.9, 
        max_tokens = 200
   )
    return response.choices[0].message.content

In [77]:
print(anwser_question_with_doc("如何提高可用性", chunks,embeddeds))

通过智能弹性集群管理优化主机群集，动态负载均衡及自动伸缩选择合适的实例类型，可有效提高系统的可用性。


In [78]:
print(anwser_question_with_doc("SpotMax是什么", chunks,embeddeds))

SpotMax是一个利用弹性计算资源进行云端系统成本优化的体系，包括相关产品及解决方案。


In [10]:
print(anwser_question_with_doc("如何防止spot instance中断", chunks,embeddeds))

通过对集群中Spot instance中断的预先补偿，可以防止其中断。
